In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional
# from google.colab import drive   #if using google colab for this program
# drive.mount('/content/drive')

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)  #dataframe display settings

In [4]:
eth_b = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Back/ETH200101-230815.csv")
eth_f = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Fwd/ETH230816-240819.csv")
#result summary for further action: low sharpe

In [5]:
sol_b = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Back/SOL200101-230815.csv")
sol_f = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Fwd/SOL230816-240819.csv")
#result summary for further action: possible
#back data sharpe >1.7, fail at fwd return

In [6]:
btc_b = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Back/BTC200101-230815.csv")
btc_f = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Fwd/BTC230816-240819.csv")
#result summary for further action: fail at w fd

In [7]:
bnb_b = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Back/BNB200101-230815.csv")
bnb_f = pd.read_csv("C:/Users/kokin/Downloads/BT_data/Fwd/BNB230816-240819.csv")
#result summary for further action: low sharpe

In [9]:
# backtesting

def backtesting(df: pd.DataFrame, window: int, long_threshold: float,short_threshold: float, plot: bool = False) -> Optional[pd.Series]:

    df= df[['Date','Close']].copy()
    df['Change']=df['Close'].pct_change()

    df['median'] = df['Close'].rolling(window).median()
    df['iqr'] = df['Close'].rolling(window).quantile(0.75) - df['Close'].rolling(window).quantile(0.25)
    df['z'] = (df['Close'] - df['median']) / df['iqr']

    df['pos'] = np.where(df['z'] > long_threshold, 1, 
                         np.where(df['z'] < -short_threshold, -1,0)) 
    #define the position given the trading logic, if z score > long threshold, position = 1, meaning to long 1 unit, vice versa 

    df['pos_t-1'] = df['pos'].shift(1)
    df['trade']=abs(df['pos_t-1'] - df['pos'])
    df['pnl'] = df['pos_t-1'] * df['Change'] - df['trade']*0.05/100   #5bps trading fee  
    df['cumu'] = df['pnl'].cumsum()
    df['dd'] = df['cumu'].cummax() - df['cumu']
    df['bnh_cumu']= df['Change'].cumsum()

    annual_return = round(df['pnl'].mean() * 365,2)

    # avoid division of zero
    if df['pnl'].std() != 0:
        sharpe = round((df['pnl'].mean() / df['pnl'].std()) * np.sqrt(365), 2)
    else:
        sharpe = np.nan

    mdd = round(df['dd'].max(),2)

    # avoid division of zero
    if mdd != 0:
        calmar = round(annual_return / mdd,2)
    else:
        calmar = np.nan

    if plot:
      fig = px.line(df, x='Date', y=['cumu','bnh_cumu'])
      fig.show()
      return

    return pd.Series([window,long_threshold,short_threshold,sharpe, calmar,annual_return,mdd],index= ['window','l_threshold','s_threshold','sharpe','calmar','AR','Mdd'])


In [ ]:
# optimisation, finding the best parameter given the above logic

window_list = np.arange(10,100,5)
l_threshold_list = np.arange(0,2.5,0.1)
s_threshold_list = np.arange(0,2.5,0.1)
result_list = []

for window in window_list:
  for lthreshold in l_threshold_list:
      for sthreshold in s_threshold_list:
        result_list.append(backtesting(df = bnb_b, window = window, long_threshold=lthreshold, short_threshold=sthreshold))
result_df = pd.DataFrame(result_list)
result_df= result_df.sort_values(by = 'sharpe', ascending = False)
result_df.head(20)

In [ ]:
# data table to find highest sharpe plateau
data_table = result_df.pivot(index= 'window' , columns= 'threshold', values= 'sharpe' )
sns.heatmap(data_table,annot=False, cmap='Greens' )
plt.show()

In [ ]:
# plot cumu return against bnh
backtesting(df = sol_f, window = 65, long_threshold = 0.3, short_threshold=1, plot = True)